In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql import Row
#from pyspark.mllib.feature import HashingTF, IDF
#from pyspark.ml.feature import HashingTF, IDF
import pandas as pd
from sklearn import metrics
import os

df = pd.read_csv('train.csv')
df.head() # broadband 即可：0-离开，1-留存
df_test = pd.read_csv('test.csv')

from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
# LabelEncoder
for col in tqdm_notebook(df.columns):
    if df[col].dtype == "object":
        encoder = LabelEncoder()
        encoder.fit(list(df[col].values) + list(df_test[col].values))
        df[col] = encoder.transform(list(df[col].values))
        df_test[col] = encoder.transform(list(df_test[col].values))
        
df = df.reset_index() # 重置索引
df_test = df_test.reset_index()
def clean_top_cols(df):
    new_cols = [col for col in df.columns if df[col].value_counts(dropna=False, normalize=True).values[0] > 0.7]
    return new_cols
df_cols = clean_top_cols(df)
df_test_clos = clean_top_cols(df_test)
cols_to_drop = list(set(df_cols + df_test_clos))
if 'bad_good' in cols_to_drop : # 查看标签列是否在其中
    cols_to_drop.remove('bad_good') # 删除标签列
#print("原始的train shape : ", df)
df = df.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)

if __name__ == "__main__":
  appname = "RandomForestClassifier"
  master ="local[4]"
  conf = SparkConf().setAppName(appname).setMaster(master) #spark配置        
  spark=SparkSession.builder.config(conf=conf).getOrCreate()#spark实例化
   
#读取数据
  #data=spark.read.csv('train.csv',header=True)

#构造训练数据集
  print("Let's go ")
  data = spark.createDataFrame(df)
  print("It is end of part")
  #dataSet = data.na.fill('0').rdd.map(list)#用0填充空值  
  #data = data.rdd.map()
  trainData, testData= data.randomSplit([0.7, 0.3], seed=7)
  trainingSet = trainData.rdd.map(lambda x:Row(label=x[4], features=Vectors.dense(x[5:3])))  
  #train_num = trainingSet.count()
  #print("训练样本数:{}".format(train_num))
  
    
#使用随机森林进行训练
  print("使用随机森林进行训练")
  stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
  si_model = stringIndexer.fit(trainingSet)
  train_tf = si_model.transform(trainingSet)
  train_tf.show(5)  
  rf = RandomForestClassifier(numTrees=100, labelCol="indexed", seed=7)
  rfModel = rf.fit(train_tf)
    
#输出模型特征重要性、子树权重
  print("模型特征重要性:{}".format(rfModel.featureImportances))
  print("模型特征数:{}".format(rfModel.numFeatures))
   
#预测测试集
  testSet = testData.map(lambda x:Row(label=x[-1], features=Vectors.dense(x[:-1]))).toDF()
  test_num=testSet.count()
  print("测试样本数:{}".format(test_num))  
  si_model = stringIndexer.fit(testSet)
  test_tf = si_model.transform(testSet)  
  predictResult = rfModel.transform(test_tf)
  predictResult.show(5)
  spark.stop()
  
#将预测结果转为python中的dataframe
  columns=predictResult.columns#提取强表字段
  predictResult=predictResult.take(test_num)#
  predictResult=pd.DataFrame(predictResult,columns=columns)#转为python中的dataframe
  
#性能评估
  y=list(predictResult['indexed'])
  y_pred=list(predictResult['prediction'])
  y_predprob=[x[1] for x in list(predictResult['probability'])]
  precision_score=metrics.precision_score(y, y_pred)#精确率
  recall_score=metrics.recall_score(y, y_pred)#召回率
  accuracy_score=metrics.accuracy_score(y, y_pred)#准确率
  f1_score=metrics.f1_score(y, y_pred)#F1分数
  auc_score=metrics.roc_auc_score(y, y_predprob)#auc分数
  print("精确率:",precision_score )#精确率
  print("召回率:",recall_score )#召回率
  print("准确率:",accuracy_score )#准确率
  print("F1分数:", f1_score)#F1分数
  print("auc分数:",auc_score )#auc分数

C:\Users\HT\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\HT\AppData\Local\Temp/ipykernel_15624/67583872.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm_notebook(df.columns):


  0%|          | 0/627 [00:00<?, ?it/s]

Let's go 
It is end of part
使用随机森林进行训练


AttributeError: 'PipelinedRDD' object has no attribute '_jdf'